#### Comparação das Planilhas

In [19]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import re

warnings.filterwarnings("ignore", category=FutureWarning)

# Abaixo adicionar o código para comparação dos dataframes
# Lista de pares de sistema e trigrama que você deseja processar
pares_para_processar = [
    ('1200', 'KFQ'),
    # Adicione mais pares conforme necessário
]

# Função para extrair sistema e trigrama do nome do arquivo
def extract_info(filename):
    pattern = r'I-LD-3010\.2S-(\d+)-\d+-(\w+)-\d+_\w+(_comments)?\.xlsx'
    match = re.search(pattern, filename)
    if match:
        return match.group(1), match.group(2)
    return None, None

# Diretório onde estão os arquivos
directory = Path('C:/Users/elxy/Documents/Codigos/Python/P84_85/LDs/files')

# Dicionário para armazenar os pares de arquivos
file_pairs = {}

# Procurar os arquivos no diretório
for file in directory.glob('*.xlsx'):
    sistema, trigrama = extract_info(file.name)
    if sistema and trigrama:
        key = (sistema, trigrama)
        if key not in file_pairs:
            file_pairs[key] = {'base': None, 'comments': None}
        
        if '_comments' in file.name:
            file_pairs[key]['comments'] = file
        else:
            file_pairs[key]['base'] = file

# Função para processar um par de arquivos
def process_file_pair(sistema, trigrama):
    key = (sistema, trigrama)
    if key not in file_pairs:
        print(f"Não foram encontrados arquivos para o sistema {sistema} e trigrama {trigrama}")
        return

    ld_file = file_pairs[key]['base']
    ld_file_comments = file_pairs[key]['comments']

    if not ld_file or not ld_file_comments:
        print(f"Par de arquivos incompleto para o sistema {sistema} e trigrama {trigrama}")
        return

    print(f"Processando - Sistema: {sistema}, Trigrama: {trigrama}")
    print(f"Arquivo base: {ld_file}")
    print(f"Arquivo de comentários: {ld_file_comments}")

    # Carregar os dataframes 
    df_LD = pd.read_excel(ld_file, sheet_name="VDRL") 
    df_LD_comments = pd.read_excel(ld_file_comments, sheet_name="VDRL") 

    # Pré-processamento dos dados 
    df_LD.columns = df_LD.iloc[6] 
    df_LD = df_LD.iloc[7:] 

    df_LD_comments.columns = df_LD_comments.iloc[6] 
    df_LD_comments = df_LD_comments.iloc[7:] 

    df_LD.rename(columns = {"CLIENT DOCUMENT NUMBER":"CLIENT_DOCUMENT", 
                            "VENDOR DOCUMENT NUMBER":"VENDOR_DOCUMENT", 
                            "DOCUMENT TITLE": "DOCUMENT_TITLE", "PLANNED DATE": "PLANNED_DATE"}, inplace = True) 
    df_LD_comments.rename(columns = {"Have comment?":"Have_comment", "CLIENT DOCUMENT NUMBER":"CLIENT_DOCUMENT", 
                                     "VENDOR DOCUMENT NUMBER":"VENDOR_DOCUMENT", 
                                     "DOCUMENT TITLE": "DOCUMENT_TITLE", "PLANNED DATE": "PLANNED_DATE"}, inplace = True) 
    df_LD = df_LD.dropna(subset=['DOCUMENT_TITLE'])
    df_LD_comments = df_LD_comments.dropna(subset=['DOCUMENT_TITLE'])

    # Remover espaços em branco e converter para maiúsculas
    df_LD['DOCUMENT_TITLE'] = df_LD['DOCUMENT_TITLE'].apply(lambda x: x.strip().upper())
    df_LD_comments['DOCUMENT_TITLE'] = df_LD_comments['DOCUMENT_TITLE'].apply(lambda x: x.strip().upper())

    colunas_para_manter = ["CLIENT_DOCUMENT", "VENDOR_DOCUMENT", "DOCUMENT_TITLE", "ORIGINATOR ", "PLANNED_DATE"]
    df_LD.drop(columns=[col for col in df_LD.columns if col not in colunas_para_manter], inplace = True)
    df_LD.reset_index(drop=True, inplace = True)

    colunas_para_manter_VDA = ["Have_comment", "Discipline", "CLIENT_DOCUMENT", "VENDOR_DOCUMENT", "DOCUMENT_TITLE", "ORIGINATOR ", "PLANNED_DATE"]
    df_LD_comments.drop(columns=[col for col in df_LD_comments.columns if col not in colunas_para_manter_VDA], inplace = True) 
    df_LD_comments.reset_index(drop=True, inplace = True)

    df_LD = df_LD.iloc[:, :-2]
    df_LD_comments = df_LD_comments.iloc[:, :-2]

    return df_LD, df_LD_comments

# Processar os pares especificados
for sistema, trigrama in pares_para_processar:
    df_LD, df_LD_comments = process_file_pair(sistema, trigrama)


Processando - Sistema: 1200, Trigrama: KFQ
Arquivo base: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\files\I-LD-3010.2S-1200-911-KFQ-001_0.xlsx
Arquivo de comentários: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\files\I-LD-3010.2S-1200-911-KFQ-001_0_comments.xlsx


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [20]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import re

warnings.filterwarnings("ignore", category=FutureWarning)

# (O restante do código permanece o mesmo até a parte de processamento dos pares)

# Processar os pares especificados
for sistema, trigrama in pares_para_processar:
    df_LD, df_LD_comments = process_file_pair(sistema, trigrama)

    # Nova célula para verificar linhas repetidas
    print(f"\nVerificando linhas repetidas - Sistema: {sistema}, Trigrama: {trigrama}")

    # Verificar linhas repetidas em df_LD
    duplicates_LD = df_LD[df_LD.duplicated(subset=['CLIENT_DOCUMENT', 'DOCUMENT_TITLE'], keep=False)]
    print(f"\nLinhas repetidas em df_LD: {len(duplicates_LD)}")
    if not duplicates_LD.empty:
        print("Linhas repetidas em df_LD:")
        print(duplicates_LD)

    # Verificar linhas repetidas em df_LD_comments
    duplicates_LD_comments = df_LD_comments[df_LD_comments.duplicated(subset=['CLIENT_DOCUMENT', 'DOCUMENT_TITLE'], keep=False)]
    print(f"\nLinhas repetidas em df_LD_comments: {len(duplicates_LD_comments)}")
    if not duplicates_LD_comments.empty:
        print("Linhas repetidas em df_LD_comments:")
        print(duplicates_LD_comments)

    # Comparação das Planilhas
    print(f'\nResultado da comparação das planilhas de COMENTADA x EMITIDA - Sistema: {sistema}, Trigrama: {trigrama}\n')
    # Comparar o número de documentos
    print(f"Número de documentos na planilha df_LD: {len(df_LD)}")
    print(f"Número de documentos na planilha df_LD_comments: {len(df_LD_comments)}")

    # Comparar as colunas "CLIENT DOCUMENT NUMBER" e "DOCUMENT TITLE"
    matches_both = 0
    matches_one = 0
    no_matches = 0
    no_match_documents = []
    one_match_documents = []

    for index, row1 in df_LD.iterrows():
        match = df_LD_comments[(df_LD_comments['CLIENT_DOCUMENT'] == row1['CLIENT_DOCUMENT']) & 
                               (df_LD_comments['DOCUMENT_TITLE'] == row1['DOCUMENT_TITLE'])]
        if not match.empty:
            matches_both += 1
        else:
            match_one = df_LD_comments[(df_LD_comments['CLIENT_DOCUMENT'] == row1['CLIENT_DOCUMENT']) | 
                                       (df_LD_comments['DOCUMENT_TITLE'] == row1['DOCUMENT_TITLE'])]
            if not match_one.empty:
                matches_one += 1
                one_match_documents.append({
                    'CLIENT_DOCUMENT_LD': row1['CLIENT_DOCUMENT'],
                    'DOCUMENT_TITLE_LD': row1['DOCUMENT_TITLE'],
                    'CLIENT_DOCUMENT_COMMENTS': match_one['CLIENT_DOCUMENT'].values[0],
                    'DOCUMENT_TITLE_COMMENTS': match_one['DOCUMENT_TITLE'].values[0]
                })
            else:
                no_matches += 1
                no_match_documents.append({
                    'CLIENT_DOCUMENT': row1['CLIENT_DOCUMENT'],
                    'DOCUMENT_TITLE': row1['DOCUMENT_TITLE']
                })

    print(f"\nDocumentos com ambas as colunas iguais: {matches_both}")
    print(f"Documentos com pelo menos uma coluna igual: {matches_one}")
    print(f"Documentos sem correspondência: {no_matches}")

    # Criar DataFrames
    df_no_match = pd.DataFrame(no_match_documents)
    df_one_match = pd.DataFrame(one_match_documents)

    # Salvar os DataFrames em um arquivo Excel
    output_file = f'documentos_comparacao_{sistema}_{trigrama}.xlsx'
    with pd.ExcelWriter(output_file) as writer:
        df_no_match.to_excel(writer, sheet_name='Sem Correspondência', index=False)
        df_one_match.to_excel(writer, sheet_name='Uma Coluna Igual', index=False)

    print(f"\nOs resultados da comparação foram salvos no arquivo: {output_file}")
    print("Aba 'Sem Correspondência': Documentos sem correspondência")
    print("Aba 'Uma Coluna Igual': Documentos com pelo menos uma coluna igual")


Processando - Sistema: 1200, Trigrama: KFQ
Arquivo base: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\files\I-LD-3010.2S-1200-911-KFQ-001_0.xlsx
Arquivo de comentários: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\files\I-LD-3010.2S-1200-911-KFQ-001_0_comments.xlsx

Verificando linhas repetidas - Sistema: 1200, Trigrama: KFQ

Linhas repetidas em df_LD: 0

Linhas repetidas em df_LD_comments: 0

Resultado da comparação das planilhas de COMENTADA x EMITIDA - Sistema: 1200, Trigrama: KFQ

Número de documentos na planilha df_LD: 255
Número de documentos na planilha df_LD_comments: 255

Documentos com ambas as colunas iguais: 255
Documentos com pelo menos uma coluna igual: 0
Documentos sem correspondência: 0

Os resultados da comparação foram salvos no arquivo: documentos_comparacao_1200_KFQ.xlsx
Aba 'Sem Correspondência': Documentos sem correspondência
Aba 'Uma Coluna Igual': Documentos com pelo menos uma coluna igual


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Gera nova lista com os comentários da versão anterior para a nova versão

Este script atualiza/gera novo arquivo: 
"C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\files\I-LD-3010.2S-5122-940-D5A-001_A_comments.xlsx"
Com base nos arquivos:
- "C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\files\I-LD-3010.2S-5122-940-D5A-001_A.xlsx" e 
- "C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\files\I-LD-3010.2S-5122-940-D5A-001_0_comments.xlsx"

_<small>OBS.: Como o arquivo recebeu I-LD-3010.2S-5122-940-D5A-001_A.xlsx atualizações e o I-LD-3010.2S-5122-940-D5A-001_0_comments.xlsx não. Então o arquivo I-LD-3010.2S-5122-940-D5A-001_A_comments.xlsx deve tomar como base o arquivo I-LD-3010.2S-5122-940-D5A-001_A.xlsx com as informações das colunas "Have comment?" "Discipline" do arquivo "I-LD-3010.2S-5122-940-D5A-001_0_comments.xlsx" quando possível. Imprimir no final uma listagem de quais informações do "I-LD-3010.2S-5122-940-D5A-001_0_comments.xlsx" não puderam ser copiadas para o arquivo "I-LD-3010.2S-5122-940-D5A-001_A.xlsx". A coluna chave para realizar o relacionamento das informações é o campo "CLIENT DOCUMENT NUMBER" nas três planilhas.</small>_

In [13]:
# Read the Excel files
base_file = directory / "I-LD-3010.2S-5266-940-ND1-001_A.xlsx"
old_comments = directory / "I-LD-3010.2S-5266-940-ND1-001_0_comments.xlsx"
new_comments = directory / "I-LD-3010.2S-5266-940-ND1-001_A_comments.xlsx"

# Read base and old comments files
df_base = pd.read_excel(base_file, sheet_name="VDRL")
df_old_comments = pd.read_excel(old_comments, sheet_name="VDRL")

# Preprocess dataframes
df_base.columns = df_base.iloc[6]
df_base = df_base.iloc[7:]
df_old_comments.columns = df_old_comments.iloc[6]
df_old_comments = df_old_comments.iloc[7:]

# Create new dataframe based on base file
df_new = df_base.copy()

# Track unmatched documents
unmatched_docs = []

# Add Have_comment and Discipline columns
df_new['Have comment?'] = None
df_new['Discipline'] = None

# Update Have_comment and Discipline based on CLIENT DOCUMENT NUMBER matches
for idx, row in df_new.iterrows():
    match = df_old_comments[df_old_comments['CLIENT DOCUMENT NUMBER'] == row['CLIENT DOCUMENT NUMBER']]
    if not match.empty:
        df_new.at[idx, 'Have comment?'] = match['Have comment?'].values[0]
        df_new.at[idx, 'Discipline'] = match['Discipline'].values[0]

# Find documents in old comments that don't exist in new file
for idx, row in df_old_comments.iterrows():
    if row['CLIENT DOCUMENT NUMBER'] not in df_new['CLIENT DOCUMENT NUMBER'].values:
        unmatched_docs.append({
            'CLIENT_DOCUMENT': row['CLIENT DOCUMENT NUMBER'],
            'DOCUMENT_TITLE': row['DOCUMENT TITLE'],
            'Have_comment': row['Have comment?'],
            'Discipline': row['Discipline']
        })

# Save the new comments file
print(f"Novos documentos com os comentários antigos gravados em {new_comments}:")
df_new.to_excel(new_comments, index=False)

if unmatched_docs:
    # Print unmatched documents
    print("\nDocuments from old comments file that couldn't be matched:")
    df_unmatched = pd.DataFrame(unmatched_docs)
    # Save unmatched documents to Excel with descriptive filename
    output_file = f'unmatched_docs_{sistema}_{trigrama}.xlsx'
    df_unmatched.to_excel(output_file, index=False)
    print(f"\nUnmatched documents saved to: {output_file}")
else:
    print("All documents were successfully matched.")
    # Criar um DataFrame vazio quando não há documentos não correspondentes
    df_unmatched = pd.DataFrame(columns=['CLIENT_DOCUMENT', 'DOCUMENT_TITLE', 'Have_comment', 'Discipline'])


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Novos documentos com os comentários antigos gravados em C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\files\I-LD-3010.2S-5266-940-ND1-001_A_comments.xlsx:

Documents from old comments file that couldn't be matched:

Unmatched documents saved to: unmatched_docs_5266_ND1.xlsx


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
